## Imports

In [2]:
from typing import Iterable, Callable
from itertools import takewhile

import numpy as np
from scipy.ndimage import gaussian_filter1d
from scipy.stats import ks_2samp
import pandas as pd
import seaborn as sns

from matplotlib.axes import Axes
import matplotlib.pyplot as plt

## Data Loading and Analysis

In [3]:
# functions
def smoothing(df: pd.DataFrame, key: str, kernel_width: float) -> np.ndarray:
    return gaussian_filter1d(df[key], sigma=kernel_width)


# FUNCTION NOT USED! KEPT HERE FOR REFERENCE IMPLEMENTATION
#
# def derivative(df: pd.DataFrame, key: str = "smoothed_fluorescence") -> np.ndarray:
#     # calculate timestep
#     time = df.time.map(lambda t: pd.to_datetime(t))
#     dt = int(np.diff(time).mean()) / 1e9  # in seconds
#
#     # calculate derivative
#     grad = np.gradient(df[key]) / dt
#     return grad


def rolling_average(arr: Iterable, width: int = 5) -> np.ndarray:
    window = np.ones(width) / width
    return np.convolve(arr, window, mode="same")


def load_namemap(platemap_filepath: str) -> dict:
    # load map of conditions to plate locations (.csv)
    platemap = (
        pd.read_csv(platemap_filepath)
        .dropna()
        .set_index("Rows/Cols")
        .T
        .unstack()
    )
    platemap = platemap[platemap != '-']

    # extract location names from platemap index
    locs = [
        "".join(loc_tup)
        for loc_tup in platemap.index
    ]
    # extract conditions from platemap values
    conditions = platemap.values

    # make dict mapping locations to conditions
    namemap = dict(zip(locs, conditions))

    return namemap


def load_experiment(
        data_filepath: str, namemap: dict, date: str,
        smoothing_parameter: float,
        rolling_average_window_length: int,
) -> pd.DataFrame:
    # load dataframe and massage
    df = (
        pd.read_csv(data_filepath)
        .drop(["Unnamed: 0", "Temperature(¡C)"], axis=1)
        .set_index("Time")
    )

    # generate dataframe for each replicate
    dict_of_dfs_raw = {
        loc: pd.DataFrame.from_dict(
            {
                "loc": loc,
                "time": df[loc].index,
                "fluorescence": df[loc].values,
                "condition": cond
            }
        )
        for loc, cond in namemap.items()
    }
    # calculate smoothed fluorescence trace for each replicate
    for _, df in dict_of_dfs_raw.items():
        df["smoothed_fluorescence"] = smoothing(df, key="fluorescence", kernel_width=smoothing_parameter)

    # calculate background fluorescence to subtract from traces
    negs = pd.concat([
        dict_of_dfs_raw[loc]
        for loc, cond in namemap.items()
        if cond == "(-)"
    ])
    background = negs.groupby("time")["smoothed_fluorescence"].mean()

    # iterate over unnormalized dict of dfs
    dict_of_dfs_processed = {}
    for loc, df in dict_of_dfs_raw.items():
        # background subtraction
        backgroud_corrected_fluorescence = df["fluorescence"] - background.values
        smoothed_fluorescence = df["smoothed_fluorescence"] - background.values

        # calculate timestep
        time = df.time.map(lambda t: pd.to_datetime(t))
        dt = int(np.diff(time).mean()) / 1e9  # in seconds

        # calculate derivative
        grad = np.gradient(smoothed_fluorescence) / dt

        # calculate rolling average of derivative
        rolling_average_derivative = rolling_average(grad, rolling_average_window_length)

        dict_of_dfs_processed[loc] = pd.DataFrame.from_dict(
            {
                "date": date,
                "loc": loc,
                "condition": df["condition"],
                "time": df["time"],
                "raw_fluorescence": df["fluorescence"],
                "bkgrd_sub_fluorescence": backgroud_corrected_fluorescence,
                "smoothed_fluorescence": smoothed_fluorescence,
                "derivative": grad,
                "smoothed_derivative": gaussian_filter1d(grad, smoothing_parameter),
                "rolling_derivative": rolling_average_derivative,
            }
        )

    # turn dictionary of processed dataframes into one big dataframe
    df = pd.concat(dict_of_dfs_processed.values()).reset_index()

    return df


def load_data(
        # date -> (platemap_filepath, data_filepath)
        experiment_filepaths: dict[str, tuple[str, str]],
        smoothing_parameter: float = 0.5,
        rolling_average_window_length: int = 5,
) -> pd.DataFrame:
    # load dataframes for each day of experiments
    list_of_dfs = [
        load_experiment(
            data_filepath=data_filepath, namemap=load_namemap(platemap_filepath), date=date,
            smoothing_parameter=smoothing_parameter,
            rolling_average_window_length=rolling_average_window_length
        )
        for date, (platemap_filepath, data_filepath) in experiment_filepaths.items()
    ]
    # concatenate list of dataframes into one big dataframe and return
    data = pd.concat(list_of_dfs).reset_index(drop=True)
    return data


def remove_outliers_timeseries(df: pd.DataFrame) -> pd.DataFrame:
    # get dataframe with single row per experimental replicate
    vmax_df = df[df["index"] == 42]

    # MANUALLY IDENTIFIED OUTLIERS!
    #
    # NOTE: we did not remove outliers in the
    #  final analysis presented in the manuscript;
    #  this code was used for dev only
    outlier_ix = {
        # 220728
        42,  # GAfast
        235, 2937,  # GAslow
    }
    # get pd.Series for each row identified by outlier_ix
    outlier_series = [vmax_df.loc[ix] for ix in outlier_ix]

    # create list of dicts of comparables (key -> Any)
    # and each
    outlier_identifiers = [
        {key: series[key] for key in "date loc".split()}  # each item represents the val of the outlier
        for series in outlier_series  # each dict represents a single outlier
    ]

    # generate masks to identify outliers by comparing df to those key, value pairs
    each_outlier_mask = []
    for outlier_dict in outlier_identifiers:
        # generate mask to find indices that match each individual key val pair
        #   for a given outlier
        each_outlier_keyval_mask = [
            df[key] == val
            for key, val in outlier_dict.items()
        ]

        # generate mask for each outlier, which is where all keyval pair
        each_outlier_mask.append(np.logical_and.reduce(each_outlier_keyval_mask))

    # generate masks for all outliers, which is where any mask is TRUE
    combined_mask = np.logical_or.reduce(each_outlier_mask)

    # return NOT outliers
    return df[~combined_mask]



## Getter Functions

In [4]:
# GETTER FUNCTIONS
def make_getter_function(conditions: Iterable[str]) -> Callable[[pd.DataFrame], pd.DataFrame]:
    def get_conditions(df: pd.DataFrame) -> pd.DataFrame:
        filtered = df[df["condition"].map(lambda cond: cond in conditions)]
        return filtered
    return get_conditions


get_experimental_conditions = make_getter_function({"GAfast", "GAslow", "(-)", "uniform", "Uniform"})
get_traces_conditions = make_getter_function({"GAfast", "GAslow", "(-)", "uniform"})
get_barplot_conditions = make_getter_function({"GAfast", "GAslow", "uniform"})
get_syn_tRNAs = make_getter_function({"GAfast", "GAslow", "uniform"})

## Plotting Functions

In [5]:
# PLOTTING PARAMETERS
hue_order = "uniform GAslow GAfast (-)".split()
cmap = dict(zip(hue_order, sns.color_palette()))
cmap["Uniform"] = cmap["uniform"]
cmap["(-)"] = (0, 0, 0)
cmap["(+)"] = (0, 0, 0)


def set_xaxis_time(ax, tight: bool = False, last_timepoint: str = "12hrs") -> Axes:
    tick_names = ["0hrs", "4hrs", "8hrs", "12hrs", "16hrs"]
    n = len(tick_names)
    tick_locs = [int(i / (n - 1) * len(ax.get_xticks())) for i in range(n)]
    tick_dict = dict(zip(tick_names, tick_locs))

    if last_timepoint is not None:
        # truncate plot
        ax.set_xlim(0, tick_dict[last_timepoint])
        # truncate tick_names and tick_locs
        tick_names, tick_locs = list(zip(*takewhile(lambda tup: tup[1] <= tick_dict[last_timepoint], tick_dict.items())))

    if tight:
        # only take first and last values of tick_names and tick_locs
        tick_names = [tick_names[0], tick_names[-1]]
        tick_locs = [tick_locs[0], tick_locs[-1]]

    ax.set_xticks(tick_locs, tick_names)

    return ax


def plot_line_seaborn(
        df: pd.DataFrame,
        xkey="time", ykey="bkgrd_sub_fluorescence",
        traces=False,
        ax=None,
        legend: bool = True,
        format_xaxis: bool = True, time_tight: bool = False,
        outpath: str = None,
) -> Axes:
    if ax is None:
        _, ax = plt.subplots()

    estimator = "mean" if traces is False else None
    sns.lineplot(
        data=df, x=xkey, y=ykey,
        hue="condition", alpha=0.5,
        palette=cmap, legend=legend,
        estimator=estimator,
        ax=ax
    )
    if ykey == "derivative":
        plt.legend(loc="lower right")

    if format_xaxis:
        ax = set_xaxis_time(ax, time_tight)

    if outpath is not None:
        plt.savefig(outpath)

    return ax


def plot_trace(
        # data analysis parameters
        plotted: pd.DataFrame,
        threshkey: str = None,
        left_threshold: float = None, right_threshold: float = None,
        # specify variables to plot
        xkey: str = "time",
        ykey: str = "bkgrd_sub_fluorescence", ystyle: str = "-",
        y2key: str = None, y2style: str = ":",
        # plotting parameters
        ax: Axes = None,
        format_xaxis: bool = True,
        plot_bounds: bool = True,                                   # plot integration bounds?
        ylim: tuple[float, float] = None,
        plot_title: str = None,
        outpath: str = None
):
    if ylim is None:
        vals = [
            val
            for key in (ykey, y2key) if key is not None
            for val in plotted[key]
        ]
        ylim = (min(vals), max(vals))

    if ax is None:
        _, ax = plt.subplots()

    cond = set(plotted["condition"]).pop()

    x = plotted[xkey]
    y1 = plotted[ykey]

    # plot data
    ax.plot(x, y1, ystyle, color=cmap[cond])

    if y2key is not None:
        y2 = plotted[y2key]
        ax.plot(x, y2, y2style, color=cmap[cond])


    if plot_bounds:
        # plot horizontal line across y=0
        ax.plot(np.zeros_like(y1), '--k')

        # get vertical lines for this trace
        vert = np.linspace(ylim[0], ylim[1], len(y1))

        # get xval arrays for vertical lines at integration bounds
        lowix, highix = get_integration_bounds(
            plotted, threshkey=threshkey,
            left_threshold=left_threshold, right_threshold=right_threshold
        )
        lowbound = np.ones_like(x) * lowix
        highbound = np.ones_like(x) * highix

        # plot integration bounds
        ax.plot(lowbound, vert, "--k")
        ax.plot(highbound, vert, "--k")

    if format_xaxis:
        ax = set_xaxis_time(ax)

    if plot_title is not None:
        ax.set_title(plot_title)

    if outpath is not None:
        plt.savefig(outpath)

    return ax


def plot_traces(
       # data analysis parameters
        plotted: pd.DataFrame,
        threshkey: str = None,
        left_threshold: float = None, right_threshold: float = None,
        # specify variables to plot
        huekey: str = "condition",
        xkey: str = "time",
        ykey: str = "bkgrd_sub_fluorescence", ystyle: str = "-",
        y2key: str = None, y2style: str = ":",
        # plotting parameters
        ax: Axes = None,
        format_xaxis: bool = True,
        plot_bounds: bool = True,                                   # plot integration bounds?
        ylim: tuple[float, float] = None,
        plot_title: str = None,
        outpath: str = None
):
    if ylim is None:
        vals = [
            val
            for key in (ykey, y2key) if key is not None
            for val in plotted[key]
        ]
        ylim = (min(vals), max(vals))

    if ax is None:
        _, ax = plt.subplots()

    for hueval, df in plotted.groupby(huekey):
        ax = plot_trace(
            plotted=df, threshkey=threshkey, left_threshold=left_threshold, right_threshold=right_threshold,
            xkey=xkey, ykey=ykey, ystyle=ystyle, y2key=y2key, y2style=y2style,
            ax=ax, format_xaxis=False, plot_bounds=plot_bounds, ylim=ylim
        )
    if plot_title is not None:
        ax.set_title(plot_title)

    if outpath is not None:
        plt.savefig(outpath)

    return ax


def plot_traces_grid(
        # data analysis parameters
        plotted: pd.DataFrame,
        threshkey: str = None, left_threshold: float = None, right_threshold: float = None,
        # specify plot layout
        rowkey: str = "condition", colkey: str = "loc",
        # specify variables to plot
        xkey: str = "time", ykey: str = "bkgrd_sub_fluorescence",
        y2key: str = None,
        # plotting parameters
        figsize: tuple[float, float] = (10, 5),
        plot_bounds: bool = True,                                   # plot integration bounds?
        plot_title: str = None,
        ylim: tuple[float, float] = None,
        tight_xaxis_labels: bool = True,
        outpath: str = None
):
    # flag if plot_bounds need to be calculated for each row
    calc_ylim = (ylim is None)

    # group data by rowkey
    list_df_by_rowkey = list(plotted.groupby(rowkey))

    # get number of valid columns per condition
    ncols_per_row = {
        row: len(set(df[colkey]))
        for row, df in list_df_by_rowkey
    }

    # get dimensions of subplots
    nrows = len(list_df_by_rowkey)
    ncols = max(ncols_per_row.values())

    fig, axes = plt.subplots(nrows, ncols, figsize=figsize)
    fig.tight_layout(h_pad=2)

    # iterate through conditions and plot
    for (row, df_by_row), ax_rows in zip(list_df_by_rowkey, axes):

        # get plot bounds if not specified
        if calc_ylim:
            vals = [
                val
                for key in (ykey, y2key) if key is not None
                for val in df_by_row[key]
            ]
            ylim = (min(vals), max(vals))

        if ncols == 1:
            ax_rows = [ax_rows]  # ax rows should be a list, but rn is just an Axes object

        for (col, df), ax in zip(df_by_row.groupby(colkey), ax_rows):
            for (_, plotted) in df.groupby("loc"):
                ax = plot_trace(
                    plotted=plotted,
                    threshkey=threshkey, left_threshold=left_threshold, right_threshold=right_threshold,
                    xkey=xkey, ykey=ykey, y2key=y2key,
                    ax=ax, plot_bounds=plot_bounds, ylim=ylim,
                    format_xaxis=False
                )

                # format
                ax.set_title(f"{row} {col}")

                ax.set_ylim(ylim)

            set_xaxis_time(ax, tight=tight_xaxis_labels)

    if plot_title is not None:
        plt.suptitle(plot_title)
        plt.subplots_adjust(top=0.85)

    if outpath is not None:
        plt.savefig(outpath)

    return fig, axes


## Calculate Translation Rates

In [6]:
def get_integration_bounds(
        df: pd.DataFrame, threshkey: str,
        left_threshold: float, right_threshold: float,
) -> tuple[int, int]:
    """
    dataframe MAY be grouped by date and condition already
    (will work on single trace dataframes)
    """
    mean_trace = df.groupby("time")[threshkey].mean()[:12 * 12]

    lowix = int(np.argmax(mean_trace >= left_threshold * max(mean_trace)))

    reverse_indices = np.arange(len(mean_trace))[::-1]
    highix = int(reverse_indices[np.argmax(mean_trace[::-1] >= right_threshold * max(mean_trace))])

    return lowix, highix


def get_average_rate(
        df: pd.DataFrame,
        bound_ix: tuple[float, float],
        integratekey: str,
        vmax=False
) -> float:
    """
    12 ~ 1 hr
    36 ~ 3 hrs
    48 ~ 4 hrs
    """
    if vmax is True:
        return max(df[integratekey])
    else:
        ix0, ix1 = bound_ix
        time = df.time.map(lambda t: pd.to_datetime(t))
        dt = int(np.diff(time).mean()) / 1e9  # in seconds
        time_window = (ix1 - ix0 + 1) * dt
        integral = df.iloc[ix0:ix1 + 1][integratekey].sum()
        integral /= time_window
        return integral


def norm_by_GAfast(data: pd.DataFrame) -> pd.DataFrame:
    def normalize(df: pd.DataFrame):
        GA_fast_mean = df[df.condition == "GAfast"]["average_rate"].mean()
        df["normalized_rate"] = df["average_rate"] / GA_fast_mean

        return df

    data = data.groupby("date").apply(normalize)
    data["elong_norm"] = 1 / data["normalized_rate"]
    return data


def get_avg_rate_df(
        df: pd.DataFrame, leftthresh: float, rightthresh: float,
        threshkey: str = "derivative", integratekey: str = "derivative", vmax=False
) -> pd.DataFrame:
    avg_rate_dicts = [
        {
            "date": date,
            "loc": loc,
            "condition": cond,
            "average_rate": get_average_rate(
                df=df,
                bound_ix=get_integration_bounds(
                    df=df, threshkey=threshkey,
                    left_threshold=leftthresh, right_threshold=rightthresh
                ),
                integratekey=integratekey,
                vmax=vmax
            )
        }
        for (date, cond), df_by_cond in df.groupby(["date", "condition"])
        for loc, df in df_by_cond.groupby("loc")
    ]
    avg_rate_series = [
        pd.Series(data=d, index=d.keys())
        for d in avg_rate_dicts
    ]
    avg_rate_df = pd.concat(avg_rate_series, axis=1).T
    return avg_rate_df


# Paper Figures

## Load Data

In [7]:
experiments = {
    # date -> (platemap_filepath, data_filepath)
    "220728": (
        "experimental_data/220728_platemap.csv", 
        "experimental_data/220728_fluorescence_data.csv"
    ),
    "220801": (
        "experimental_data/220801_platemap.csv", 
        "experimental_data/220801_fluorescence_data.csv"
    ),
    "221207": (
        "experimental_data/221207_platemap.csv", 
        "experimental_data/221207_fluorescence_data.csv"
    )
}

sigma=10
left_thresh = 0.50
right_thresh = 0.50
thresh_key = "smoothed_derivative"
integrate_key = "smoothed_derivative"
barplotval = "normalized_rate"
vmax_flag = False


data_df = load_data(experiment_filepaths=experiments, smoothing_parameter=sigma)
no_outliers = remove_outliers_timeseries(data_df)
rate_df = norm_by_GAfast(
    get_avg_rate_df(
        get_barplot_conditions(no_outliers),
        threshkey=thresh_key, integratekey=integrate_key,
        leftthresh=left_thresh, rightthresh=right_thresh,
        vmax=vmax_flag
    )
)
rate_w_outliers = norm_by_GAfast(
    get_avg_rate_df(
        get_barplot_conditions(data_df),
        threshkey=thresh_key, integratekey=integrate_key,
        leftthresh=left_thresh, rightthresh=right_thresh,
        vmax=vmax_flag
    )
)

FileNotFoundError: [Errno 2] No such file or directory: '220728_platemap.csv'

## Plot translation rate bar graph

In [ ]:
plotted = get_barplot_conditions(rate_w_outliers)

ax = sns.stripplot(
    data=plotted, x=barplotval, y="condition",
    palette=cmap, size=10, linewidth=2
)
ax = sns.barplot(
    data=plotted, x=barplotval, y="condition",
    hue_order=hue_order, palette=cmap, orient="h",
    ax=ax
)

# plt.savefig(f"../figures/rates_bar_plot.svg")

In [ ]:
df = plotted
df["normalized_rate"] = df["normalized_rate"].astype("float")
df.groupby(["date", "condition"])["normalized_rate"].describe()

In [ ]:
plotted["normalized_rate"] = plotted["normalized_rate"].astype(float)
plotted.groupby("condition")["normalized_rate"].describe()

## 220801 representative fluorescence vs time traces

In [ ]:
df = get_traces_conditions(data_df)
plotted = df[df["date"] == "220801"]
means = plotted.groupby(["condition", "time"])["smoothed_fluorescence"].mean().reset_index()
ax = plot_line_seaborn(plotted, ykey="bkgrd_sub_fluorescence", format_xaxis=False, legend=True)
ax = plot_traces(
    plotted=means,
    ykey="smoothed_fluorescence", ystyle=":",
    ax=ax, format_xaxis=False,
    plot_bounds=False
)
set_xaxis_time(ax, last_timepoint="12hrs")
ax.set_ylim([-5000, 30000])
# plt.savefig("../figures/220801_raw_traces.svg")

## Mean Derivative vs Condition

In [ ]:
df = get_traces_conditions(data_df)
plotted = df[df["date"]=="220801"]
plot_line_seaborn(
    df=plotted, ykey="smoothed_derivative", 
    # outpath="../figures/220801_mean_derivative_vs_condition.svg",
)

## fluorescence vs day 

In [ ]:
plotted = get_barplot_conditions(no_outliers)
# plotted = get_barplot_conditions(data_df)
plot_traces_grid(
    plotted=plotted, 
    threshkey=thresh_key, left_threshold=left_thresh, right_threshold=right_thresh,
    rowkey="date", colkey="condition",
    ykey="smoothed_fluorescence", 
    y2key="bkgrd_sub_fluorescence",
    plot_bounds=False,
    tight_xaxis_labels=False,
    figsize=(10,8), 
    # outpath="../graph_gazing/derivatives_vs_loc.png"
)

## derivatives vs day 

In [ ]:
plotted = get_barplot_conditions(no_outliers)
# plotted = get_barplot_conditions(data_df)
plot_traces_grid(
    plotted=plotted, 
    threshkey=thresh_key, left_threshold=left_thresh, right_threshold=right_thresh,
    rowkey="date", colkey="condition",
    ykey="derivative", 
    # y2key="bkgrd_sub_fluorescence",
    plot_bounds=True,
    tight_xaxis_labels=False,
    figsize=(10,8), 
    # outpath="../graph_gazing/derivatives_vs_loc.png"
)

## 221207 derivatives vs well location

In [ ]:
df = no_outliers
# df = data_df
plotted = get_barplot_conditions(df[df["date"] == "221207"])
plot_traces_grid(
    plotted=plotted, 
    threshkey=thresh_key, left_threshold=left_thresh, right_threshold=right_thresh,
    rowkey="condition", colkey="loc",
    ykey="derivative", 
    # y2key="derivative",
    plot_bounds=False,
    tight_xaxis_labels=False,
    # figsize=(10,8), outpath="../graph_gazing/221207_derivative_vs_loc.svg"
)

## Integration Parameter Sweep

In [ ]:
def correlate_bounds(thresh: str):
    bounds_set = {
        "low_ten_percent": (0.1, 0.1),
        "low_quartile": (0.25, 0.25),
        "halfmax": (0.5, 0.5),
        "high_quartile": (0.75, 0.75),
        "high_ten_percent": (0.9, 0.9),
        "low_high": (0.1, 0.9),
        "zero_bounds": (0,0)
    }
    rate_dfs = {
        bound_name: get_barplot_conditions(norm_by_GAfast(
            get_avg_rate_df(
                get_barplot_conditions(no_outliers),
                threshkey=threshkey,
                leftthresh=left, rightthresh=right
            )
        ))["average_rate"]
        for bound_name, (left, right) in bounds_set.items()
    }
    rate_vs_bounds_df = pd.DataFrame.from_dict(rate_dfs)
    corr = rate_vs_bounds_df.astype("float").corr(method="spearman")
    sns.heatmap(data=corr, cbar="viridis", vmin=0.6)
    outpath = f"../graph_gazing/{threshkey}_corr_heatmap"
    # UNCOMMENT TO SAVE TO FILE
    # plt.savefig(f"{outpath}.svg")
    # plt.savefig(f"{outpath}.png")
    

### Calculating from smoothed derivative 

In [ ]:
threshkey="smoothed_derivative"
correlate_bounds(threshkey)

### Calculating from raw derivative

In [ ]:
threshkey="derivative"
correlate_bounds(threshkey)

# Analysis within Day

## raw traces

In [ ]:
figsize = (5,10)

traces = get_traces_conditions(data_df)
fig, axes = plt.subplots(3,1, figsize=figsize)
fig.tight_layout(h_pad=4)
means = traces.groupby(["condition", "date", "time"])["smoothed_fluorescence"].mean().reset_index()

for i, ((date, traces_df), (date, means_df), ax) in enumerate(zip(
    traces.groupby("date"), means.groupby("date"), axes
)):
    plot_line_seaborn(
        df=traces_df, ykey="bkgrd_sub_fluorescence", 
        ax=ax, format_xaxis=False, legend=True,
    )
    plot_traces(
        plotted=means_df,
        ykey="smoothed_fluorescence", ystyle=":",
        ax=ax, format_xaxis=False,
        plot_bounds=False
    )
    set_xaxis_time(ax, last_timepoint="12hrs")
    ax.set_title(f"tRNA Batch #{i+1}")
# plt.savefig("../figures/all_raw_traces.svg")

## derivatives

In [ ]:
figsize = (5,10)

traces = get_traces_conditions(data_df)
fig, axes = plt.subplots(3,1, figsize=figsize)
fig.tight_layout(h_pad=4)

for i, ((date, traces_df),  ax) in enumerate(zip(
    traces.groupby("date"), axes
)):
    plot_line_seaborn(
        df=traces_df, ykey="smoothed_derivative", 
        ax=ax,
    )

    ax.set_title(f"tRNA Batch #{i+1}")
# plt.savefig("../figures/all_derivatives.svg")

## bar plots

In [ ]:
figsize = (5,10)

bars = get_barplot_conditions(rate_w_outliers)
fig, axes = plt.subplots(3,1, figsize=figsize)
fig.tight_layout(h_pad=4)

for i, ((date, bars_df),  ax) in enumerate(zip(bars.groupby("date"), axes)):
    ax = sns.stripplot(
        data=bars_df, x=barplotval, y="condition",
        palette=cmap, size=10, linewidth=2, ax=ax
    )
    ax = sns.barplot(
        data=bars_df, x=barplotval, y="condition",
        hue_order=hue_order, palette=cmap, orient="h",
        ax=ax
    )
    
    ax.set_title(f"tRNA Batch #{i+1}")
    
    cond_dict = {cond: df["normalized_rate"] for cond, df in bars_df.groupby("condition")}
    stat = ks_2samp(cond_dict["GAfast"], cond_dict["GAslow"], alternative="less")
    print(f"Experiment #{i+1}: ks 2 sample one sided test: p = {stat.pvalue:.2%}")
# plt.savefig("../figures/all_bars.svg")

In [ ]:
bars_df["normalized_rate"] = bars_df["normalized_rate"].astype(float)
bars_df.groupby("condition")["normalized_rate"].describe()

In [ ]:
stat = ks_2samp(cond_dict["uniform"], cond_dict["GAslow"])
print(f"Experiment #{i+1}: ks 2 sample two sided test: p = {stat.pvalue:.2%}")

In [ ]:
just_two_experiments = bars[bars["date"] != "220728"]
cond_dict = {cond: df["normalized_rate"] for cond, df in just_two_experiments.groupby("condition")}
stat = ks_2samp(cond_dict["GAfast"], cond_dict["GAslow"], alternative="less")
print(f"Pooled Experiments 2 and 3: ks 2 sample one sided test: p = {stat.pvalue:.3%}")